<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [12]</a>'.</span>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import yfinance as yf
import datetime

from valueinvesting import utils as utils

# 1. Load raw input
## Hardcoded block

In [2]:
evaluate_last_X_years = True
X=10
currency_pair = 'NOK=X'
numbers_in_currency = 'NOK'
share_name = 'BONHR.OL'

# 1.2. Raw input statement data

In [3]:
route = f"../data/input/{share_name}_data.csv"
data = pd.read_csv(route, sep=';', parse_dates=['date', 'real_date'])
# replace -, extend missing 0-s, fix datatype
data = utils.replace_format_input(data)

In [4]:
owned_shares = pd.read_csv('../data/extras/owned_stocks.csv', sep=';', parse_dates=['date'])
owned_shares = owned_shares.loc[(owned_shares['stock'] == share_name) & (owned_shares['active'] == 'yes')]

## (OPTIONAL) Time Filtering - Last 7 years daat

In [5]:
if evaluate_last_X_years:
    data = data[data['date'] > datetime.datetime.today() - datetime.timedelta(days=X*366+93)]
    data = data.reset_index(drop=True)

data.head()

,date,revenue,cogs,gross_profit,net_profit,cash,acc_rec,inventory,curr_assets,goodwill,...,acc_pay,short_term_debt,current_debt,curr_liab,long_term_debt,total_liab,cash_from_operating_activities,capex,shares,real_date
0,2014-06-30,2771,0,2771,130,5862,0,811,9007,0,...,0,0,2588,7835,14197,28917,990,-1363,32.3,2014-07-15
1,2014-09-30,3411,0,3411,-38,6741,0,872,10079,0,...,0,0,1886,6751,16568,30486,290,-1237,32.3,2014-10-25
2,2014-12-31,3440,0,3440,273,5673,0,995,9250,0,...,0,0,1148,7409,17602,33485,1202,-1219,32.3,2015-02-14
3,2015-03-31,3707,0,3707,614,5501,0,1152,9374,0,...,0,0,1995,8551,17531,35532,-2032,5736,32.3,2015-05-05
4,2015-06-30,3918,0,3918,-2390,8952,0,1111,12987,0,...,0,0,3370,10468,17300,35884,1789,-1271,32.3,2015-07-15


## Calculate date when quaterly reports could be available

In [6]:
data = utils.calculate_real_date(data)

# 2. Filter out unnecesary columns

In [7]:
data = utils.filter_raw_data(data)
data.tail()

,date,real_date,shares,revenue,cogs,gross_profit,net_profit,cash,acc_rec,inventory,...,intangible_assets,total_assets,acc_pay,short_term_debt,current_debt,curr_liab,long_term_debt,total_liab,cash_from_operating_activities,capex
36,2023-06-30,2023-07-15,42.50,2834,0,2834,555,6222,0,406,...,1250,24510,0,0,1569,5146,9421,17824,465,-251
37,2023-09-30,2023-10-25,42.50,3197,0,3197,172,5998,0,408,...,1186,23462,0,0,2413,5889,8202,16905,348,-248
38,2023-12-31,2024-02-14,42.50,3531,0,3531,287,5460,0,549,...,1223,23476,0,0,2363,5997,7717,16799,1,-294
39,2024-03-31,2024-04-29,42.53,2973,0,2973,304,6545,2839,556,...,1261,24581,0,0,2209,6166,8275,16377,1337,-342
40,2024-06-30,2024-07-08,42.53,4283,0,4283,694,6802,2715,392,...,1255,24222,0,0,2167,6164,7662,15759,1313,-243


## Pull historical average USD national currency rates and add to the dataframe

In [8]:
data['usd_nat_currency'] = utils.daily_price(
    ticker=currency_pair,
    end=data['date'],
    days_earlier=90
    )

# drop rows, when USD rates wasn't available
data = data[data['usd_nat_currency'].notna()]

data.tail()

,date,real_date,shares,revenue,cogs,gross_profit,net_profit,cash,acc_rec,inventory,...,total_assets,acc_pay,short_term_debt,current_debt,curr_liab,long_term_debt,total_liab,cash_from_operating_activities,capex,usd_nat_currency
36,2023-06-30,2023-07-15,42.50,2834,0,2834,555,6222,0,406,...,24510,0,0,1569,5146,9421,17824,465,-251,10.699984
37,2023-09-30,2023-10-25,42.50,3197,0,3197,172,5998,0,408,...,23462,0,0,2413,5889,8202,16905,348,-248,10.483742
38,2023-12-31,2024-02-14,42.50,3531,0,3531,287,5460,0,549,...,23476,0,0,2363,5997,7717,16799,1,-294,10.821103
39,2024-03-31,2024-04-29,42.53,2973,0,2973,304,6545,2839,556,...,24581,0,0,2209,6166,8275,16377,1337,-342,10.502364
40,2024-06-30,2024-07-08,42.53,4283,0,4283,694,6802,2715,392,...,24222,0,0,2167,6164,7662,15759,1313,-243,10.750878


## Convert USD values related to IS, BS, CFS to national currency

In [9]:
# convert columns into national currency if necessary
if numbers_in_currency == 'USD':
    data_nat_curr = utils.convert_national_currency(input_data=data, currency=data)
else:
    data_nat_curr = data.copy()
data_nat_curr.tail()

,date,real_date,shares,revenue,cogs,gross_profit,net_profit,cash,acc_rec,inventory,...,total_assets,acc_pay,short_term_debt,current_debt,curr_liab,long_term_debt,total_liab,cash_from_operating_activities,capex,usd_nat_currency
36,2023-06-30,2023-07-15,42.50,2834,0,2834,555,6222,0,406,...,24510,0,0,1569,5146,9421,17824,465,-251,10.699984
37,2023-09-30,2023-10-25,42.50,3197,0,3197,172,5998,0,408,...,23462,0,0,2413,5889,8202,16905,348,-248,10.483742
38,2023-12-31,2024-02-14,42.50,3531,0,3531,287,5460,0,549,...,23476,0,0,2363,5997,7717,16799,1,-294,10.821103
39,2024-03-31,2024-04-29,42.53,2973,0,2973,304,6545,2839,556,...,24581,0,0,2209,6166,8275,16377,1337,-342,10.502364
40,2024-06-30,2024-07-08,42.53,4283,0,4283,694,6802,2715,392,...,24222,0,0,2167,6164,7662,15759,1313,-243,10.750878


# 3. Calculate input to Value Raios (eps, bv per share, fcf)
## filter out unwanted columns

In [10]:
filtered_nat_curr = utils.calculate_input_value_ratios(data_nat_curr)
filtered_nat_curr.head()

,date,real_date,shares,revenue,cogs,gross_profit,net_profit,cash,acc_rec,inventory,...,usd_nat_currency,net_profit_ttm,cash_from_operating_activities_ttm,capex_ttm,revenue_ttm,cogs_ttm,eps,bv_per_share,fcf,fcf_per_share
0,2014-06-30,2014-07-15,32.3,2771,0,2771,130,5862,0,811,...,5.978844,NaN,NaN,NaN,NaN,None,NaN,218.482972,NaN,NaN
1,2014-09-30,2014-10-25,32.3,3411,0,3411,-38,6741,0,872,...,6.240275,NaN,NaN,NaN,NaN,None,NaN,223.467492,NaN,NaN
2,2014-12-31,2015-02-14,32.3,3440,0,3440,273,5673,0,995,...,6.876861,NaN,NaN,NaN,NaN,None,NaN,245.541796,NaN,NaN
3,2015-03-31,2015-05-05,32.3,3707,0,3707,614,5501,0,1152,...,7.736785,979.0,450.0,1917.0,13329.0,None,30.309598,265.448916,-1467.0,-45.417957
4,2015-06-30,2015-07-15,32.3,3918,0,3918,-2390,8952,0,1111,...,7.752235,-1541.0,1249.0,2009.0,14476.0,None,-47.708978,272.879257,-760.0,-23.529412


## Calculate input values to value ratios

In [11]:
ratios_nat_curr = utils.ratios_input_filter(filtered_nat_curr)
ratios_nat_curr.head()

,date,real_date,revenue,eps,bv_per_share,shares,fcf,fcf_per_share,cash,total_liab,revenue_ttm,cogs_ttm
0,2014-06-30,2014-07-15,2771,NaN,218.482972,32.3,NaN,NaN,5862,28917,NaN,None
1,2014-09-30,2014-10-25,3411,NaN,223.467492,32.3,NaN,NaN,6741,30486,NaN,None
2,2014-12-31,2015-02-14,3440,NaN,245.541796,32.3,NaN,NaN,5673,33485,NaN,None
3,2015-03-31,2015-05-05,3707,30.309598,265.448916,32.3,-1467.0,-45.417957,5501,35532,13329.0,None
4,2015-06-30,2015-07-15,3918,-47.708978,272.879257,32.3,-760.0,-23.529412,8952,35884,14476.0,None


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [12]:
ratios_nat_curr = utils.evaluate_performance(input=filtered_nat_curr, output=ratios_nat_curr)
ratios_nat_curr.head()

ZeroDivisionError: float division by zero

# 4. Pull historical share prices and merge with sattement data

In [ ]:
# pull weekly share prices and merge with the value ratios
merged_nat_curr = utils.add_share_prices_to_value_ratios(share_name, data, ratios_nat_curr)
merged_nat_curr.head()

# Calculate price ratios (P/E, P/B, P/S, P/FCF)

In [ ]:
merged_nat_curr = utils.price_ratios(merged_nat_curr)
merged_nat_curr.head()

# Calculate Dividend yield and currency-share price correlation

In [ ]:
# get daily historical USD-national currency rates
usd_nat_curr = utils.get_historical_currency_rate(currency_pair, merged_nat_curr)

In [ ]:
# get share prices and dividend yield data
ticker_all_price = utils.get_historical_share_dividend(share_name, merged_nat_curr)

In [ ]:
# Pearson's correlation coefficient between share price and USD - national currency rate
utils.get_currency_share_price_correlation(share_name, usd_nat_curr, ticker_all_price)

# Share price and value ratios

In [ ]:
for column in merged_nat_curr.columns[3:]:
    fig, ax1 = plt.subplots(figsize=(15, 6))

    ax2 = ax1.twinx()
    ax1.plot(merged_nat_curr['real_date'], merged_nat_curr[column], color='k', label=merged_nat_curr)
    ax2.plot(ticker_all_price.index, ticker_all_price['Close'], color='b', label=share_name)

    ax1.set_xlabel('Date')
    ax1.set_ylabel(column, color='k')
    ax2.set_ylabel('Share price (national currency)', color='b')

    plt.legend()
    plt.show()

In [ ]:
# plot histograms of value parameters
utils.plot_histogram_value_parameters(input_df=merged_nat_curr, extra_parameters=[], owned_shares=owned_shares)

## Calculate correlation between value parameters

In [ ]:
# calulate value parameters Pearson's correlation coefficient
pd.DataFrame(merged_nat_curr, columns=['roa', 'roe', 'pe_ratio', 'pb_ratio', 'ps_ratio', 'ev_revenue', 'debt_to_equity', 'current_ratio']).corr()

In [ ]:
final_df = utils.get_value_stock_target_prices(
    input_df=merged_nat_curr,
    min_pct=0.2,
    max_pct=0.5,
    owned_shares=owned_shares
)